In [1]:
import cv2
import numpy as np
import scipy.spatial.distance as distance
from sklearn.preprocessing import normalize
import pandas as pd
import time
from sklearn.cluster import KMeans

In [ ]:
database = []
for image in range(1, 257):
    descriptor = np.loadtxt("data/sifts/sift1to256/im" + str(image) + ".txt")
    database.append(descriptor)
print(database[0])

In [ ]:
def BoW(requests, database, clusters):
    K = clusters.shape[0]
    kmeans_model = KMeans(n_clusters=K)
    kmeans_model.cluster_centers_ = clusters
    results = []
    for request in requests:
        results.append(kmeans_model.predict(database[request].reshape(1, -1)))
results = BoW(requests, nbow, k8)